In [268]:
import numpy as np
import pandas as pd
from collections import defaultdict

import requests
from bs4 import BeautifulSoup
import re
import os
import time
import csv

import matplotlib.pyplot as plt
%matplotlib inline

In [269]:
apikey = open('apikey.txt', 'r').read()
db = 'pmc'
base = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?'
pmcid = 'PMC3463246'
s = '{:s}db={:s}&id={:s}'.format(base, db, pmcid, apikey)

In [270]:
out = requests.get(s)
bs = BeautifulSoup(out.content, 'lxml')
bs.prettify;

In [271]:
### Get article metadata

title = bs.find('title-group').find('article-title').text
abstract = bs.abstract.text

# Article ids
pub_id_type = [i['pub-id-type'] for i in bs.findAll('article-id')]
pub_id_nr = [i.text for i in bs.findAll('article-id')]
pub_ids = list(zip(pub_id_type, pub_id_nr))

# Dates
date_tags = [i['pub-type'] for i in bs.findAll('pub-date')]
date_years = [i.year.text for i in bs.findAll('pub-date')]
dates = list(zip(date_tags, date_years))

# Author list and affiliations
names = [i.text for i in bs.find('contrib-group').findAll('given-names')]
surnames = [i.text for i in bs.find('contrib-group').findAll('surname')]
authors = list(zip(names, surnames))
affiliations = [i.text for i in bs.findAll('aff')]  # TODO: exclude <label># 

# Article keywords
keywords = [i.text for i in bs.find('kwd-group').findAll('kwd')]

# Journal
journal_title = bs.find('journal-title-group').find('journal-title').text

# Check if full text available
full_text_available = bs.findAll('sec') == []  # TODO: find a better way to check

In [281]:
print('Full text available?:', full_text_available)

Full text available?: False


In [279]:
print(journal_title)

eLife


In [273]:
print(title)

A bacterial sulfonolipid triggers multicellular development in the closest living relatives of animals


In [274]:
print(abstract)


Bacterially-produced small molecules exert profound influences on animal health, morphogenesis, and evolution through poorly understood mechanisms. In one of the closest living relatives of animals, the choanoflagellate Salpingoeca rosetta, we find that rosette colony development is induced by the prey bacterium Algoriphagus machipongonensis and its close relatives in the Bacteroidetes phylum. Here we show that a rosette inducing factor (RIF-1) produced by A. machipongonensis belongs to the small class of sulfonolipids, obscure relatives of the better known sphingolipids that play important roles in signal transmission in plants, animals, and fungi. RIF-1 has extraordinary potency (femtomolar, or 10−15 M) and S. rosetta can respond to it over a broad dynamic range—nine orders of magnitude. This study provides a prototypical example of bacterial sulfonolipids triggering eukaryotic morphogenesis and suggests molecular mechanisms through which bacteria may have contributed to the evoluti

In [280]:
print(keywords)

['Salpingoeca rosetta', 'Algoriphagus', 'bacterial sulfonolipid', 'multicellular development']


In [275]:
print(pub_ids)

[('pmid', '23066504'), ('pmc', '3463246'), ('publisher-id', '00013'), ('doi', '10.7554/eLife.00013'), ('doi', '10.7554/eLife.00013.029'), ('doi', '10.7554/eLife.00013.030')]


In [276]:
print(dates)

[('epub', '2012'), ('collection', '2012')]


In [277]:
print(authors)

[('Rosanna A', 'Alegado'), ('Laura W', 'Brown'), ('Shugeng', 'Cao'), ('Renee K', 'Dermenjian'), ('Richard', 'Zuzow'), ('Stephen R', 'Fairclough'), ('Jon', 'Clardy'), ('Nicole', 'King')]


In [278]:
print(affiliations)

['1Department of Molecular and Cell Biology, University of California, Berkeley, Berkeley, United States', '2Department of Biological Chemistry and Molecular Pharmacology, Harvard Medical School, Boston, United States', '3Department of Biochemistry, Stanford University School of Medicine, Stanford, United States', 'University of Washington, United States', 'University of Washington, United States']
